In [3]:
import pandas as pd
import re

df = pd.read_csv("../data/processed/wnba_2025_cleaned.csv")

# add NAME_KEY
def normalize_name(name):
    name = str(name).upper()
    name = re.sub(r"[^A-Z]", "", name)
    return name

# define a function to calculate player score
def calc_score(pts, stl, ast, tov):
    return pts * 0.55 + stl * 0.22 + ast * 0.05 - tov * 0.18

# add a SCORE column
df["SCORE"] = df.apply(
    lambda row: calc_score(row["PTS"], row["STL"], row["AST"], row["TOV"]), 
    axis=1
)
#add a NAME-KEY column
df["NAME_KEY"] = df["PLAYER_NAME"].apply(normalize_name)

# out put csv file
output_path = "../data/processed/wnba_2025_player_scores.csv"
df.to_csv(output_path, index=False)

print(df)

         PLAYER_NAME  PTS  STL  AST  TOV   SCORE        NAME_KEY
0        A'ja Wilson  937   64  122   87  519.87       AJAWILSON
1    Aaliyah Edwards  194   23   20   41  105.38  AALIYAHEDWARDS
2        Aaliyah Nye  169   13   24   27   92.15      AALIYAHNYE
3      Aari McDonald  196   25   94   46  109.72    AARIMCDONALD
4      Aerial Powers   74    8   20   11   41.48    AERIALPOWERS
..               ...  ...  ...  ...  ...     ...             ...
177     Tina Charles  700   33   74   82  381.20     TINACHARLES
178    Tyasha Harris   23    2   13    3   13.20    TYASHAHARRIS
179  Veronica Burton  525   50  265   94  296.08  VERONICABURTON
180  Yvonne Anderson    0    2    2    0    0.54  YVONNEANDERSON
181        Zia Cooke   90   10   14   22   48.44        ZIACOOKE

[182 rows x 7 columns]
